In [1]:
import os
from dotenv import load_dotenv
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
import requests
import re
import pandas as pd

In [2]:
load_dotenv()

API_KEY = os.getenv("PERPLEXITY_API_KEY")
if not API_KEY:
    raise ValueError("Missing or empty PERPLEXITY_API_KEY environment variable")

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "accept": "application/json",
}

In [3]:
def perplexity_query(messages):
    url = "https://api.perplexity.ai/chat/completions"
    json_data = {
        "model": "sonar",
        "messages": messages,
        "temperature": 0.4,
        "max_tokens": 500,
    }
    response = requests.post(url, headers=headers, json=json_data, timeout=30)
    response.raise_for_status()
    data = response.json()
    return data["choices"][0]["message"]["content"]

In [4]:
df = pd.read_csv("sensor_data.csv", parse_dates=["timestamp"])

In [5]:
print("Preview of 5 entries in sensor_data.csv:")
print(df.head())

Preview of 5 entries in sensor_data.csv:
   id  soil_moisture  temperature  wind_speed  wind_direction  light  \
0   1          33.02         27.1         5.2             326    379   
1   2          18.59         29.3        10.4             176    670   
2   3          13.30         22.5        10.6             134    423   
3   4          15.51         31.1         3.0             352    529   
4   5          21.60         28.2         2.9             330    330   

   uv_index  humidity  pressure  leaf_moisture           timestamp  
0       6.1        49       998             13 2025-11-10 15:05:26  
1       3.7        49      1002             19 2025-11-10 15:35:26  
2       5.7        50       996             13 2025-11-10 16:05:26  
3       6.5        71      1013             46 2025-11-10 16:35:26  
4       2.1        50       988             19 2025-11-10 17:05:26  


In [6]:
class State(TypedDict):
    question: str
    data_summary: str  
    result: str
    answer: str

In [7]:
def generate_data_summary(state: State) -> State:
    recent = df[df["timestamp"] >= pd.Timestamp.now() - pd.Timedelta(days=1)]
    soil_moisture_avg = recent["soil_moisture"].mean()
    temp_avg = recent["temperature"].mean()
    humidity_avg = recent["humidity"].mean()
    summary = (
        f"Recent soil moisture average: {soil_moisture_avg:.2f}%, "
        f"temperature average: {temp_avg:.1f}C, "
        f"humidity average: {humidity_avg:.1f}%."
    )
    state["data_summary"] = summary
    return state

In [8]:
def run_dummy_query(state: State) -> State:
    state["result"] = state.get("data_summary", "")
    return state

In [9]:
def generate_decision(state: State) -> State:
    messages = [
        {"role": "system", "content": """

# 🌾 System Prompt: Agricultural Decision Agent

## **Role**

You are an agentic AI designed to support farmers and agricultural technicians.
Your primary task is to deliver **clear Yes or No answers** to user questions about **agricultural actions** (e.g., watering, fertilizing, pest control, harvesting).
Your answers are always based on **real-time sensor data** and ** perplexity API query results**.

---

## **Behavior**

* Always respond **only** with “✅ Yes” or “❌ No” — unless explicitly asked for an explanation.
* Maintain a **neutral, professional, and factual** tone.
* When data is missing, respond with:

  > ⚠️ “Data insufficient — unable to determine.”
* Never speculate or guess.

---

## **Data Context**

You have access to:

* **Environmental sensors** (e.g., soil moisture, temperature, humidity, rainfall, pH, nutrient levels).
* **External APIs** for weather forecasts, crop growth stages, and pest detection.

Always interpret sensor and API data before giving a decision.

---

## **Decision Logic**

1. **Retrieve Data:** Query sensors and APIs relevant to the user’s question.
2. **Analyze:** Compare results against agronomic thresholds (e.g., ideal soil moisture ranges).
3. **Decide:** Output a single-word decision — **Yes** or **No**.
4. **Fallback:** If inputs conflict or are missing → output “Data insufficient — unable to determine.”

---

## **Output Format**

Always respond in **Markdown**.

**Example 1:**

> User: Should I water the corn today?
> **Agent:** ✅ Yes

**Example 2:**

> User: Should I spray pesticides now?
> **Agent:** ❌ No

**Example 3:**

> User: Should I apply nitrogen fertilizer?
> **Agent:** ⚠️ Data insufficient — unable to determine.

---

## **Tools**

### 🌡️ df

Use this tool to retrieve live readings from field sensors.
Input: Sensor type(s) and location.
Output: JSON data of sensor metrics.

### Perplexity Sonar API

Use this tool to query the Perplexity Sonar API.
Input: User's agriculture question.
Output: Decision on what the user should do in ONE SENTENCE/WORD based on sensor data and querying the Sonar API.
---

## **Constraints**

* Do **not** provide reasoning or data summaries unless explicitly requested.
* Do **not** offer advice beyond a binary decision.
* Be efficient: minimize API calls by only querying what’s relevant to the user’s question.

"""},
        {"role": "user", "content": f"Sensor data summary: {state['data_summary']}\nQuestion: {state['question']}"}
    ]
    answer = perplexity_query(messages)
    state["answer"] = answer
    return state

In [10]:
graph = StateGraph(State)
graph.add_node("generate_data_summary", generate_data_summary)
graph.add_node("dummy_query", run_dummy_query)
graph.add_node("generate_decision", generate_decision)
graph.add_edge(START, "generate_data_summary")
graph.add_edge("generate_data_summary", "dummy_query")
graph.add_edge("dummy_query", "generate_decision")
graph.add_edge("generate_decision", END)

In [11]:
agent = graph.compile()

In [12]:
initial_state = {
    "question": "Should I irrigate my wheat crops grown in well-draining loam soil today based on current soil moisture and weather?",
    "data_summary": "",
    "result": "",
    "answer": "",
}

In [13]:
result_state = agent.invoke(initial_state)

In [14]:
print(f"Final decision: {result_state['answer']}")

Final decision: ❌ No
